<a href="https://colab.research.google.com/github/zubinsharma17/Alzhiem/blob/main/Alzhiem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This error is because of the un-initialized camera.

In [ ]:
import cv2
import numpy as np

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(0)

# Initialize the dataset
face_data = []
count = 0

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Draw the bounding box around the detected faces and save the face data
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Extract the face region of interest (ROI)
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        # Save the face ROI in the dataset
        face_data.append(face_section)
        count += 1

    # Display the output frame
    cv2.imshow('Face Detection', frame)

    # Press 'q' to quit the program and save the dataset
    if cv2.waitKey(1) == ord('q'):
        break

# Convert the face dataset to a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print(face_data.shape)

# Save the face dataset as a numpy array
np.save('face_data.npy', face_data)
print("Dataset saved successfully...")

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


error: ignored

The upper code will save a file named "face_data.npy" in the numpy format.
this will then be loaaded in the following code.

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load the face dataset
face_data = np.load('face_data.npy')

# Load the labels for each face in the dataset
face_labels = np.arange(face_data.shape[0])
face_labels = face_labels.reshape((face_labels.shape[0], 1))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(face_data, face_labels, test_size=0.2, random_state=42)

# Train a linear regression model on the training set
reg = LinearRegression().fit(X_train, y_train)

# Evaluate the model on the testing set
score = reg.score(X_test, y_test)
print("Accuracy:", score)

# Initialize the camera
cap = cv2.VideoCapture(0)

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Recognize the faces in the frame using the trained model
    for (x, y, w, h) in faces:
        # Extract the face region of interest (ROI)
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        # Convert the face ROI to a 1D array
        face_section = face_section.reshape((1, -1))

        # Predict the label for the face using the trained model
        label = int(reg.predict(face_section)[0][0])

        # Draw the bounding box around the recognized face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, str(label), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the output frame
    cv2.imshow('Face Recognition', frame)

    # Press 'q' to quit the program
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


To add the names to the face which will be recognized we will import a new file named "names.npy". These names will be shown on the face.
Note: In this code, the names array should be a 1-dimensional numpy array containing the names of the people associated with the face data in the same order as the data.

In [ ]:
import cv2
import numpy as np

# Load the face detection model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(0)

# Initialize the dataset
face_data = []
count = 0

# Load the saved face data
face_data = np.load('face_data.npy')

# Load the saved names associated with the face data
names = np.load('names.npy')

# Train the linear regression model
model = cv2.face.LBPHFaceRecognizer_create()
model.train(face_data, np.array(names))

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Draw the bounding box around the detected faces and predict the names
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Extract the face region of interest (ROI)
        offset = 10
        face_section = gray[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        # Predict the name of the face using the trained model
        pred = model.predict(face_section)

        # Draw the name of the person on the face
        cv2.putText(frame, names[pred[0]], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow('Face Recognition', frame)

    # Press 'q' to quit the program
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


## **This code only detects faces from the camera and saves them in a dataset. It does not include face recognition functionality. To recognize a face, you would need to train a machine learning model using the dataset of faces you have collected. Once the model is trained, it can then be used to recognize faces in real-time from the camera and label them with their corresponding names. The labeling of the name can be done by adding text to the output frame using the cv2.putText function.